In [ ]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import root_mean_squared_error
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, KBinsDiscretizer, RobustScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import BayesianRidge
from sklearn.base import BaseEstimator, RegressorMixin, ClassifierMixin, clone
import os
import joblib
import itertools
import logging
import optuna

from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

from src.feature_engineering import *
from src.modeling import *

import warnings
warnings.filterwarnings("ignore")

In [ ]:
#config
DATA_PATH = "data/"
OUTPUT_PATH = "output/"
BLUEPRINT_PATH = "output/blueprint/"
MODEL_PATH = "models/"

PREPROCESSING_LIST = {
    'outlier_removal' : {
        'enabled' : True
    },
    'robust_scaler'  : {
        'enabled' : True
    },
    'polynomial' : {
        'enabled' : False
    },
    'binning' : {
        'enabled' : False
    },
    'pca' : {
        'enabled' : True
    }
}

In [ ]:
# --- Setup logging ---
logging.basicConfig(
    filename=os.path.join(OUTPUT_PATH, "08_optuna_optimization.log"),
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)
logging.info(f"starting optuna optimization...")

In [ ]:
logging.info("reading train and test data...")

df_train = pd.read_csv(DATA_PATH + "train.csv")
# df_train = df_train.sample(n=10000).reset_index(drop=True)

df_test = pd.read_csv(DATA_PATH + "test.csv")

target_col = "BeatsPerMinute"
feature_cols = [f for f in df_train.columns if f not in ('id', target_col)]

X_train = df_train[feature_cols].copy().reset_index(drop=True)
y_train = df_train[target_col].copy().reset_index(drop=True)

logging.info(f"X_train shape : {X_train.shape}")
logging.info(f"y_train shape : {y_train.shape}")

X_test = df_test[feature_cols].copy().reset_index(drop=True)

logging.info(f"X_test shape : {X_test.shape}")

In [ ]:
logging.info("defining preprocessing combinations...")

available_steps = []

if PREPROCESSING_LIST["outlier_removal"]["enabled"]:
    available_steps.append(("outlier_removal", OutlierRemoval()))

if PREPROCESSING_LIST["robust_scaler"]["enabled"]:
    available_steps.append(("robust_scaler", RobustScaler()))

if PREPROCESSING_LIST["polynomial"]["enabled"]:
    available_steps.append(("polynomial", PolynomialFeatures(interaction_only=False, include_bias=False, degree=2)))

if PREPROCESSING_LIST["binning"]["enabled"]:
    available_steps.append(("binning", KBinsDiscretizer(encode="ordinal")))

# if PREPROCESSING_LIST["standardization"]["enabled"]:
#     available_steps.append(("standardization", StandardScaler()) 

if PREPROCESSING_LIST["pca"]["enabled"]:
    available_steps.append(("pca", PCA(n_components=0.85)))

def generate_all_combinations(steps):
    for r in range(1, len(steps) + 1):
        for combo in itertools.combinations(steps, r):
            yield combo

preprocessing_combinations = list(generate_all_combinations(available_steps))

logging.info(f"preprocessing combination count : {len(preprocessing_combinations)}")

BayesianRidge

In [ ]:
logging.info("starting BayesianRidge optuna optimization...")

def objective(trial, preprocessing_steps, X, y, n_splits=5):
    # Hyperparameter suggestions
    tol = trial.suggest_float("tol", 1e-8, 1e-2, log=True)
    alpha_1 = trial.suggest_float("alpha_1", 1e-9, 1e3, log=True)
    alpha_2 = trial.suggest_float("alpha_2", 1e-9, 1e3, log=True)
    lambda_1 = trial.suggest_float("lambda_1", 1e-9, 1e3, log=True)
    lambda_2 = trial.suggest_float("lambda_2", 1e-9, 1e3, log=True)
    fit_intercept = trial.suggest_categorical("fit_intercept", [True, False])

    model = Pipeline(
        preprocessing_steps + [
            ("regressor", BayesianRidge(
                tol=tol,
                alpha_1=alpha_1,
                alpha_2=alpha_2,
                lambda_1=lambda_1,
                lambda_2=lambda_2,
                fit_intercept=fit_intercept,
                compute_score=False,
                verbose=False
            ))
        ]
    )

    cv = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    scores = cross_val_score(model, X, y, cv=cv, scoring="neg_root_mean_squared_error", n_jobs=-1)
    return -scores.mean()

model_name = "bayesian_ridge"
results = []
for combo in preprocessing_combinations:
    steps = []
    steps_name = []
    for step_name, transformer in combo:
        steps.append((step_name, transformer))
        steps_name.append(step_name)

    study = optuna.create_study(direction="minimize")
    study.optimize(lambda trial:objective(trial, steps, X_train, y_train), n_trials=30)

    logging.info(f"model_name: {model_name}. steps : {','.join(steps_name)}. best params: {study.best_params}. best rmse: {study.best_value}")
    
    results.append({
        'model' : model_name,
        'steps' : ','.join(steps_name),
        'best_params' : study.best_params,
        'best_rmse' : study.best_value,
    })

    df_results = pd.DataFrame(results).sort_values('best_rmse', ascending=True)
    #save temporary results to csv
    df_results.to_csv(os.path.join(OUTPUT_PATH, f"08_optuna_optimization_{model_name}_results.csv"), index=False)

In [ ]:
df_results.head(10)

LGBM

In [ ]:
logging.info("starting LGBM optuna optimization...")

def objective(trial, preprocessing_steps, X, y, n_splits=5):
    # Hyperparameter suggestions
    param = {
        "num_leaves": trial.suggest_int("num_leaves", 16, 256),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2, log=True),
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "min_child_samples": trial.suggest_int("min_child_samples", 10, 50),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        "objective": "regression",
        "random_state": 42,
        "n_jobs": 1,
        "verbose": -1,
    }

    model = Pipeline(
        preprocessing_steps + [
            ("regressor", LGBMRegressor(**param))
        ]
    )

    cv = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    scores = cross_val_score(model, X, y, cv=cv, scoring="neg_root_mean_squared_error", n_jobs=-1)
    return -scores.mean()

model_name = "lgbm"
results = []
for combo in preprocessing_combinations:
    steps = []
    steps_name = []
    for step_name, transformer in combo:
        steps.append((step_name, transformer))
        steps_name.append(step_name)

    study = optuna.create_study(direction="minimize")
    study.optimize(lambda trial:objective(trial, steps, X_train, y_train), n_trials=30)

    logging.info(f"model_name: {model_name}. steps : {','.join(steps_name)}. best params: {study.best_params}. best rmse: {study.best_value}")
    
    results.append({
        'model' : model_name,
        'steps' : ','.join(steps_name),
        'best_params' : study.best_params,
        'best_rmse' : study.best_value,
    })

    df_results = pd.DataFrame(results).sort_values('best_rmse', ascending=True)
    #save temporary results to csv
    df_results.to_csv(os.path.join(OUTPUT_PATH, f"08_optuna_optimization_{model_name}_results.csv"), index=False)

In [ ]:
df_results.head(10)

Random Forest

In [ ]:
logging.info("starting Random Forest optuna optimization...")

def objective(trial, preprocessing_steps, X, y, n_splits=5):
    # Hyperparameter suggestions
    param = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 30),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 20),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 10),
        "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2", 0.5, 0.8, 1.0]),
        "bootstrap": trial.suggest_categorical("bootstrap", [True, False]),
        "random_state": 42,
        "n_jobs": 1,
    }

    model = Pipeline(
        preprocessing_steps + [
            ("regressor", RandomForestRegressor(**param))
        ]
    )

    cv = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    scores = cross_val_score(model, X, y, cv=cv, scoring="neg_root_mean_squared_error", n_jobs=-1)
    return -scores.mean()

model_name = "rf"
results = []
for combo in preprocessing_combinations:
    steps = []
    steps_name = []
    for step_name, transformer in combo:
        steps.append((step_name, transformer))
        steps_name.append(step_name)

    study = optuna.create_study(direction="minimize")
    study.optimize(lambda trial:objective(trial, steps, X_train, y_train), n_trials=30)

    logging.info(f"model_name: {model_name}. steps : {','.join(steps_name)}. best params: {study.best_params}. best rmse: {study.best_value}")
    
    results.append({
        'model' : model_name,
        'steps' : ','.join(steps_name),
        'best_params' : study.best_params,
        'best_rmse' : study.best_value,
    })

    df_results = pd.DataFrame(results).sort_values('best_rmse', ascending=True)
    #save temporary results to csv
    df_results.to_csv(os.path.join(OUTPUT_PATH, f"08_optuna_optimization_{model_name}_results.csv"), index=False)

In [ ]:
df_results.head(10)

XGB

In [ ]:
logging.info("starting XGB optuna optimization...")

def objective(trial, preprocessing_steps, X, y, n_splits=5):
    # Hyperparameter suggestions
    param = {
        "booster": trial.suggest_categorical("booster", ["gbtree", "dart"]),  
        "lambda": trial.suggest_loguniform("lambda", 1e-8, 10.0),   # L2 regularization
        "alpha": trial.suggest_loguniform("alpha", 1e-8, 10.0),    # L1 regularization
        "subsample": trial.suggest_uniform("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_uniform("colsample_bytree", 0.5, 1.0),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-4, 0.3),
        "n_estimators": trial.suggest_int("n_estimators", 100, 2000),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "gamma": trial.suggest_loguniform("gamma", 1e-8, 10.0),
        "objective": "reg:squarederror",
        "random_state": 42,
        "n_jobs": 1,
        "verbose": -1,
    }

    model = Pipeline(
        preprocessing_steps + [
            ("regressor", XGBRegressor(**param))
        ]
    )

    cv = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    scores = cross_val_score(model, X, y, cv=cv, scoring="neg_root_mean_squared_error", n_jobs=-1)
    return -scores.mean()

model_name = "xgb"
results = []
for combo in preprocessing_combinations:
    steps = []
    steps_name = []
    for step_name, transformer in combo:
        steps.append((step_name, transformer))
        steps_name.append(step_name)

    study = optuna.create_study(direction="minimize")
    study.optimize(lambda trial:objective(trial, steps, X_train, y_train), n_trials=30)

    logging.info(f"model_name: {model_name}. steps : {','.join(steps_name)}. best params: {study.best_params}. best rmse: {study.best_value}")
    
    results.append({
        'model' : model_name,
        'steps' : ','.join(steps_name),
        'best_params' : study.best_params,
        'best_rmse' : study.best_value,
    })

    df_results = pd.DataFrame(results).sort_values('best_rmse', ascending=True)
    #save temporary results to csv
    df_results.to_csv(os.path.join(OUTPUT_PATH, f"08_optuna_optimization_{model_name}_results.csv"), index=False)

In [ ]:
df_results.head(10)

CatBoost

In [ ]:
logging.info("starting CatBoost optuna optimization...")

def objective(trial, preprocessing_steps, X, y, n_splits=5):
    # Hyperparameter suggestions
    param = {
        "iterations": trial.suggest_int("iterations", 500, 5000),
        "depth": trial.suggest_int("depth", 4, 10),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.3, log=True),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1e-3, 10.0, log=True),
        # "bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 10.0),
        "random_strength": trial.suggest_float("random_strength", 0.0, 10.0),
        "border_count": trial.suggest_int("border_count", 32, 255),
        "grow_policy": trial.suggest_categorical("grow_policy", ["SymmetricTree", "Depthwise", "Lossguide"]),
        "bootstrap_type": trial.suggest_categorical("bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]),
        "random_state": 42,
    }

    model = Pipeline(
        preprocessing_steps + [
            ("regressor", CatBoostRegressor(**param))
        ]
    )

    cv = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    scores = cross_val_score(model, X, y, cv=cv, scoring="neg_root_mean_squared_error", n_jobs=-1)
    return -scores.mean()

model_name = "catboost"
results = []
for combo in preprocessing_combinations:
    steps = []
    steps_name = []
    for step_name, transformer in combo:
        steps.append((step_name, transformer))
        steps_name.append(step_name)

    study = optuna.create_study(direction="minimize")
    study.optimize(lambda trial:objective(trial, steps, X_train, y_train), n_trials=30)

    logging.info(f"model_name: {model_name}. steps : {','.join(steps_name)}. best params: {study.best_params}. best rmse: {study.best_value}")
    
    results.append({
        'model' : model_name,
        'steps' : ','.join(steps_name),
        'best_params' : study.best_params,
        'best_rmse' : study.best_value,
    })

    df_results = pd.DataFrame(results).sort_values('best_rmse', ascending=True)
    #save temporary results to csv
    df_results.to_csv(os.path.join(OUTPUT_PATH, f"08_optuna_optimization_{model_name}_results.csv"), index=False)

In [ ]:
df_results.head(10)